<a href="https://colab.research.google.com/github/zanzaneamit/amitz_DL/blob/master/Assignment_4_Python_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4 Questions

## Problem Statement

Q.1 Given a sequence of n values x1, x2, ..., xn and a window size k>0, the k-th moving average of
the given sequence is defined as follows:

The moving average sequence has n-k+1 elements as shown below.
The moving averages with k=4 of a ten-value sequence (n=10) is shown below

    i 1 2 3 4 5 6 7 8 9 10
    ===== == == == == == == == == == ==
    Input 10 20 30 40 50 60 70 80 90 100
    y1 25 = (10+20+30+40)/4
    y2 35 = (20+30+40+50)/4
    y3 45 = (30+40+50+60)/4
    y4 55 = (40+50+60+70)/4
    y5 65 = (50+60+70+80)/4
    y6 75 = (60+70+80+90)/4
    y7 85 = (70+80+90+100)/4

Thus, the moving average sequence has n-k+1=10-4+1=7 values.

Write a function to find moving average in an array over a window:

Test it over [3, 5, 7, 2, 8, 10, 11, 65, 72, 81, 99, 100, 150] and window of 3.

In [0]:
def moving_avg(list1,window):
  mov_avg = []
  for i in range(len(list1) - window + 1):
    sum = 0
    for j in range(i, window + i):
      sum = sum + list1[j]
    mov_avg.append(sum/window)
  return mov_avg

l1 = list(map(int, input('Input values for sequence: ').split(',')))
w = int(input('Input window: '))

print(moving_avg(l1,w))




Input values for sequence: 3, 5, 7, 2, 8, 10, 11, 65, 72, 81, 99, 100, 150
Input window: 3
[5.0, 4.666666666666667, 5.666666666666667, 6.666666666666667, 9.666666666666666, 28.666666666666668, 49.333333333333336, 72.66666666666667, 84.0, 93.33333333333333, 116.33333333333333]


In [0]:
import pandas as pd
df = pd.DataFrame(data = [3, 5, 7, 2, 8, 10, 11, 65, 72, 81, 99, 100, 150])
w = 3
df['mov_avg'] = df.rolling(w).mean().fillna(0)
print(df)

      0     mov_avg
0     3    0.000000
1     5    0.000000
2     7    5.000000
3     2    4.666667
4     8    5.666667
5    10    6.666667
6    11    9.666667
7    65   28.666667
8    72   49.333333
9    81   72.666667
10   99   84.000000
11  100   93.333333
12  150  116.333333


Q.2 How-to-count-distance-to-the-previous-zero
For each value, count the difference back to the previous zero (or the start of the Series,
whichever is closer)
create a new column 'Y'
Consider a DataFrame df where there is an integer column 'X'
    
    import pandas as pd
    df = pd.DataFrame({'X': [7, 2, 0, 3, 4, 2, 5, 0, 3, 4]})

In [0]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'X': [7, 2, 0, 3, 4, 2, 5, 0, 3, 4]})
s1 = pd.Series(data=df.index[df['X'] == 0])
s1=pd.Series(data=-1).append(s1)
print(s1)
s2=s1.shift(-1,fill_value=10000)

df['ind'] = df.index
i,j = np.where((df['ind'].values[:,None] >= s1.values) & (df['ind'].values[:,None] < s2.values))
print(i,j)
pd_new = pd.DataFrame(np.column_stack([df.values[i],s1.values[j]]),columns=['X','ind','diff'])
pd_new['dist_from_prev_zero'] = pd_new['ind'] - pd_new['diff']
print(pd_new)

0   -1
0    2
1    7
dtype: int64
[0 1 2 3 4 5 6 7 8 9] [0 0 1 1 1 1 1 2 2 2]
   X  ind  diff  dist_from_prev_zero
0  7    0    -1                    1
1  2    1    -1                    2
2  0    2     2                    0
3  3    3     2                    1
4  4    4     2                    2
5  2    5     2                    3
6  5    6     2                    4
7  0    7     7                    0
8  3    8     7                    1
9  4    9     7                    2


In [0]:
!pip install pandasql
import pandas as pd
import pandasql as ps

df = pd.DataFrame({'X': [7, 2, 0, 3, 4, 2, 5, 0, 3, 4]})
df['ind'] = df.index

s1 = pd.Series(data=df.index[df['X'] == 0])
s1=pd.Series(data=-1).append(s1)
print(s1)
s2=s1.shift(-1,fill_value=10000)
new_df = {'range_from': s1, 'range_to': s2}
df_range = pd.DataFrame(new_df)
print(df_range)

sqlcode = '''
select *
from df
inner join df_range on df.ind >= df_range.range_from
and df.ind < df_range.range_to
'''

newdf = ps.sqldf(sqlcode,locals())
newdf['dist_from_prev_zero'] = newdf['ind'] - newdf['range_from']
print(newdf)

0   -1
0    2
1    7
dtype: int64
   range_from  range_to
0          -1         2
0           2         7
1           7     10000
   X  ind  range_from  range_to  dist_from_prev_zero
0  7    0          -1         2                    1
1  2    1          -1         2                    2
2  0    2           2         7                    0
3  3    3           2         7                    1
4  4    4           2         7                    2
5  2    5           2         7                    3
6  5    6           2         7                    4
7  0    7           7     10000                    0
8  3    8           7     10000                    1
9  4    9           7     10000                    2


Q.3 Create a DatetimeIndex that contains each business day of 2015 and use it to index a Series of
random numbers.

In [0]:
import pandas as pd
import numpy as np

dt1 = pd.bdate_range(start = '2015-01-01',end = '2015-12-31')
data1 = pd.Series(data=np.random.randint(1,1000,len(dt1)),index=dt1)
print(data1)



2015-01-01    776
2015-01-02    893
2015-01-05    246
2015-01-06    154
2015-01-07     72
             ... 
2015-12-25    740
2015-12-28    753
2015-12-29    959
2015-12-30    356
2015-12-31    896
Freq: B, Length: 261, dtype: int64


Q.4 